In [2]:
import pandas as pd
import numpy as np

In [4]:
# 对于数值数据，pandas使用浮点值NaN表示缺失数据，称为哨兵值，可以方便地检测出来
string_data = pd.Series( ['aardvark', 'artichoke', np.nan, 'avocado'] )
string_data

0     aardvark
1    artichoke
2          NaN
3      avocado
dtype: object

In [5]:
string_data.isnull()

0    False
1    False
2     True
3    False
dtype: bool

In [6]:
# python内置的None值在对象数组中也可以作为NA
string_data[0] = None
string_data.isnull()

0     True
1    False
2     True
3    False
dtype: bool

In [7]:
# 滤除缺失数据
# 可以通过isnull或布尔索引的手工方法，但dropna可能会更实用一些
#  对于一个Series，dropna返回一个仅含非空数据和(原)索引值的Series
from numpy import nan as NA
data = pd.Series( [1, NA, 3.5, NA, 7] )
data.dropna()

0    1.0
2    3.5
4    7.0
dtype: float64

In [9]:
# 这等价于
data[ data.notnull() ]

0    1.0
2    3.5
4    7.0
dtype: float64

In [11]:
# 对于DataFrame对象，dropna默认丢弃任何含有缺失值的行
data = pd.DataFrame([[1., 6.5, 3.], [1., NA, NA],
                     [NA, NA, NA], [NA, 6.5, 3.]] )
cleaned = data.dropna()
data

,0,1,2
0,1.0,6.5,3.0
1,1.0,NaN,NaN
2,NaN,NaN,NaN
3,NaN,6.5,3.0


In [12]:
cleaned

,0,1,2
0,1.0,6.5,3.0


In [13]:
# 传入how='all'将只丢弃全为NA的那些行
data.dropna(how = 'all')

,0,1,2
0,1.0,6.5,3.0
1,1.0,NaN,NaN
3,NaN,6.5,3.0


In [14]:
# 用这种方式丢弃列，只需传入axis = 1
data

,0,1,2
0,1.0,6.5,3.0
1,1.0,NaN,NaN
2,NaN,NaN,NaN
3,NaN,6.5,3.0


In [15]:
data[4]=NA

In [16]:
data

,0,1,2,4
0,1.0,6.5,3.0,NaN
1,1.0,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN
3,NaN,6.5,3.0,NaN


In [17]:
data.dropna( axis=1, how='all' )

,0,1,2
0,1.0,6.5,3.0
1,1.0,NaN,NaN
2,NaN,NaN,NaN
3,NaN,6.5,3.0


In [18]:
# 另一个滤除DataFrame行的问题涉及时间序列数据。
# 假设你只想留下一部分观测数据，可以用thresh参数实现此目的
df = pd.DataFrame( np.random.randn(7, 3) )
df.iloc[:4, 1] = NA
df.iloc[:2, 2] = NA
df

,0,1,2
0,0.659304,NaN,NaN
1,1.253298,NaN,NaN
2,-0.497886,NaN,-0.535227
3,1.540409,NaN,0.756206
4,2.511189,0.714968,-0.373624
5,-0.198237,0.978615,-0.779192
6,-0.496898,-0.033586,-1.124609


In [19]:
df.dropna()

,0,1,2
4,2.511189,0.714968,-0.373624
5,-0.198237,0.978615,-0.779192
6,-0.496898,-0.033586,-1.124609


In [20]:
df.dropna(thresh=2)# 至少两个不是缺失值

,0,1,2
2,-0.497886,NaN,-0.535227
3,1.540409,NaN,0.756206
4,2.511189,0.714968,-0.373624
5,-0.198237,0.978615,-0.779192
6,-0.496898,-0.033586,-1.124609


In [21]:
# 填充缺失数据
# fillna方法是最主要的函数，通过一个常数调用fillna就会将缺失值替换为那个常数值
df.fillna(0)

,0,1,2
0,0.659304,0.000000,0.000000
1,1.253298,0.000000,0.000000
2,-0.497886,0.000000,-0.535227
3,1.540409,0.000000,0.756206
4,2.511189,0.714968,-0.373624
5,-0.198237,0.978615,-0.779192
6,-0.496898,-0.033586,-1.124609


In [22]:
# 若是通过一个字典调用fillna，就可以实现对不同的列填充不同的值
df.fillna( {1: 0.5, 2: 0} )

,0,1,2
0,0.659304,0.500000,0.000000
1,1.253298,0.500000,0.000000
2,-0.497886,0.500000,-0.535227
3,1.540409,0.500000,0.756206
4,2.511189,0.714968,-0.373624
5,-0.198237,0.978615,-0.779192
6,-0.496898,-0.033586,-1.124609


In [23]:
# fillna默认会返回新对象，但也可以对现有对象就地修改
_ = df.fillna(0, inplace = True )
df

,0,1,2
0,0.659304,0.000000,0.000000
1,1.253298,0.000000,0.000000
2,-0.497886,0.000000,-0.535227
3,1.540409,0.000000,0.756206
4,2.511189,0.714968,-0.373624
5,-0.198237,0.978615,-0.779192
6,-0.496898,-0.033586,-1.124609


In [26]:
# 对reindexing有效的那些插值方法也可用于fillna
df = pd.DataFrame( np.random.randn(6, 3) )
df.iloc[2:, 1] = NA
df.iloc[4:, 2] = NA
df

,0,1,2
0,-0.003935,-1.899053,-1.624956
1,0.135626,1.343935,-0.195934
2,-1.334782,NaN,-0.488053
3,-0.187139,NaN,-0.996919
4,-1.085732,NaN,NaN
5,1.952662,NaN,NaN


In [27]:
df.fillna(method = 'ffill')

,0,1,2
0,-0.003935,-1.899053,-1.624956
1,0.135626,1.343935,-0.195934
2,-1.334782,1.343935,-0.488053
3,-0.187139,1.343935,-0.996919
4,-1.085732,1.343935,-0.996919
5,1.952662,1.343935,-0.996919


In [28]:
df.fillna( method = 'ffill', limit = 2 )

,0,1,2
0,-0.003935,-1.899053,-1.624956
1,0.135626,1.343935,-0.195934
2,-1.334782,1.343935,-0.488053
3,-0.187139,1.343935,-0.996919
4,-1.085732,NaN,-0.996919
5,1.952662,NaN,-0.996919


In [29]:
# 只要有些创新，你就可以利用fillna实现许多别的功能。
# 比如说，你可以传入Series的平均值或中位数
data = pd.Series([1., NA, 3.5, NA, 7])
data.fillna( data.mean() )

0    1.000000
1    3.833333
2    3.500000
3    3.833333
4    7.000000
dtype: float64

In [2]:
import pandas as pd
import numpy as np

In [3]:
# 移除重复数据
data = pd.DataFrame( {'k1': ['one', 'two']*3 + ['two'],
                      'k2': [1, 1, 2, 3, 3, 4, 4]} )
data

,k1,k2
0,one,1
1,two,1
2,one,2
3,two,3
4,one,3
5,two,4
6,two,4


In [4]:
# DataFrame的duplicated方法返回一个布尔型的Series，表示各行是否是重复行(前面出现过)
data.duplicated()

0    False
1    False
2    False
3    False
4    False
5    False
6     True
dtype: bool

In [5]:
# 还有一个与此相关的drop_duplicates方法，他会返回一个DataFrame，重复的数组会标为False
data.drop_duplicates()

,k1,k2
0,one,1
1,two,1
2,one,2
3,two,3
4,one,3
5,two,4


In [6]:
# 这两个方法默认会判断全部列，你也可以指定部分列进行重复项判断
data['v1'] = range(7)
data.drop_duplicates(['k1'])

,k1,k2,v1
0,one,1,0
1,two,1,1


In [7]:
# duplicated和drop_duplicates默认保留的是第一个出现的值组合。传入keep= 'last'则保留最后一个
data.drop_duplicates(['k1', 'k2'], keep = 'last')

,k1,k2,v1
0,one,1,0
1,two,1,1
2,one,2,2
3,two,3,3
4,one,3,4
6,two,4,6


In [9]:
# 利用函数或映射进行数据转换
data = pd.DataFrame( {'food': ['bacon', 'pulled pork', 'bacon',
                               'Pastrami', 'corned beef', 'Bacon',
                               'pastrami', 'honey ham', 'nova lox'],
                      'ounces': [4, 3, 12, 6, 7.5, 8, 3, 5, 6]} )
data

,food,ounces
0,bacon,4.0
1,pulled pork,3.0
2,bacon,12.0
3,Pastrami,6.0
4,corned beef,7.5
5,Bacon,8.0
6,pastrami,3.0
7,honey ham,5.0
8,nova lox,6.0


In [16]:
# 假设你想要添加一列表示该肉类食物来源的动物类型。先编写一个不同肉类到动物的映射
meat_to_animal = {'bacon': 'pig',
                  'pulled pork': 'pig',
                  'pastrami': 'cow',
                  'corned beef': 'cow',
                  'honey ham': 'pig',
                  'nova lox': 'salmon'}
meat_to_animal

{'bacon': 'pig',
 'pulled pork': 'pig',
 'pastrami': 'cow',
 'corned beef': 'cow',
 'honey ham': 'pig',
 'nova lox': 'salmon'}

In [11]:
# Series的map方法可以接受一个函数或含有映射关系的字典型对象
# 但是这里有一个小问题，即有些肉类的首字母大写了，而另一些则没有。
# 因此，我们还需要使用Series的str.lower()方法
lowercased = data['food'].str.lower()

In [12]:
lowercased

0          bacon
1    pulled pork
2          bacon
3       pastrami
4    corned beef
5          bacon
6       pastrami
7      honey ham
8       nova lox
Name: food, dtype: object

In [17]:
data['animal'] = lowercased.map(meat_to_animal)
data

,food,ounces,animal
0,bacon,4.0,pig
1,pulled pork,3.0,pig
2,bacon,12.0,pig
3,Pastrami,6.0,cow
4,corned beef,7.5,cow
5,Bacon,8.0,pig
6,pastrami,3.0,cow
7,honey ham,5.0,pig
8,nova lox,6.0,salmon


In [18]:
# 我们也可以传入一个能够完成全部这些工作的函数
data['food'].map( lambda x: meat_to_animal[x.lower()] )
data

,food,ounces,animal
0,bacon,4.0,pig
1,pulled pork,3.0,pig
2,bacon,12.0,pig
3,Pastrami,6.0,cow
4,corned beef,7.5,cow
5,Bacon,8.0,pig
6,pastrami,3.0,cow
7,honey ham,5.0,pig
8,nova lox,6.0,salmon


In [19]:
# 用replace替换值
data = pd.Series([1., -999., 2., -999., -1000, 3.])
data

0       1.0
1    -999.0
2       2.0
3    -999.0
4   -1000.0
5       3.0
dtype: float64

In [20]:
# -999这个值可能是一个表示缺失数据的标记值。要将其替换为pandas能够理解的NA值
# 我们可以利用replace来产生一个新的Series（除非传入inplace=True）
data.replace(-999, np.nan)

0       1.0
1       NaN
2       2.0
3       NaN
4   -1000.0
5       3.0
dtype: float64

In [21]:
# 如果你希望一次性替换多个值，可以传入一个由待替换值组成的列表以及一个替换值
data.replace([-999, -1000], np.nan)

0    1.0
1    NaN
2    2.0
3    NaN
4    NaN
5    3.0
dtype: float64

In [22]:
# 要让每个值有不同的替换值，可以传递一个替换列表
data.replace([-999, -1000], [np.nan, 0])

0    1.0
1    NaN
2    2.0
3    NaN
4    0.0
5    3.0
dtype: float64

In [23]:
# 传入的参数也可以是字典
data.replace( {-999: np.nan, -1000: 0} )

0    1.0
1    NaN
2    2.0
3    NaN
4    0.0
5    3.0
dtype: float64

In [24]:
# 轴标签也可以通过函数或映射进行转换，从而得到一个新的不同的标签对象
data = pd.DataFrame(np.arange(12).reshape((3, 4)),
                    index = ['Ohio', 'Colorado', 'New York'],
                    columns = ['one', 'two', 'three', 'four'])
data

,one,two,three,four
Ohio,0,1,2,3
Colorado,4,5,6,7
New York,8,9,10,11


In [25]:
# 跟Series一样，轴索引也有一个map方法
transform = lambda x: x[:4].upper()
data.index.map(transform)

Index(['OHIO', 'COLO', 'NEW '], dtype='object')

In [27]:
data.index = data.index.map(transform)
data

,one,two,three,four
OHIO,0,1,2,3
COLO,4,5,6,7
NEW,8,9,10,11


In [28]:
# 如果想要创建数据的转换版（而不是修改原始数据），比较实用的方法是rename
data.rename(index = str.title, columns = str.upper)

,ONE,TWO,THREE,FOUR
Ohio,0,1,2,3
Colo,4,5,6,7
New,8,9,10,11


In [29]:
# Rename可以结合字典型对象实现对部分轴标签的更新
data.rename(index = {'OHIO': 'INDIANA'},
            columns = {'three': 'peekaboo'})

,one,two,peekaboo,four
INDIANA,0,1,2,3
COLO,4,5,6,7
NEW,8,9,10,11


In [30]:
# rename可以实现复制DataFrame并对其索引和列标签进行赋值
data.rename(index = {'OHIO': 'INDIANA'}, inplace = True)
data

,one,two,three,four
INDIANA,0,1,2,3
COLO,4,5,6,7
NEW,8,9,10,11
